In [1]:
start= 1
start

1

In [216]:
#!/usr/bin/python3
    
# IMPORTNAT  this code accept only .fa file as proteome

 
def parse_oma_db(oma_database_address):
    
    ############### Parsing OMA db ####################
    ###################################################

    oma_db = db.Database(oma_database_address)

    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time, "- OMA data is parsed and its release name is:", oma_db.get_release_name())
    list_oma_speices = [z.uniprot_species_code for z in oma_db.tax.genomes.values()] 
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time,"- There are",len(list_oma_speices),"species in the OMA database.")
    
    return (oma_db, list_oma_speices)


def parse_proteome(list_oma_speices):
    
    ############### Parsing query proteome of species #######
    #########################################################

    
    # IMPORTNAT  this code accept only .fa file as proteome
    project_files = listdir(address_working_folder+"/omamer_search/proteome/")

    query_species_names = []
    for file in project_files:
        if file.split(".")[-1] == "fa":
            file_name_split = file.split(".")[:-1]
            query_species_names.append('.'.join(file_name_split))
        if file.split(".")[-1] == "fasta":
            file_name_split = file.split(".")[:-1]
            query_species_names.append('.'.join(file_name_split))

    # we may assert existence of query_species_name+".fa/hogmap"
    prots_record_allspecies = [ ]
    for query_species_name in query_species_names:
        prot_address = address_working_folder +"omamer_search/proteome/"+ query_species_name + ".fa" 
        
        prots_record = list(SeqIO.parse(prot_address, "fasta")) 
        prots_record_allspecies.append(prots_record)

    query_species_num = len(query_species_names)    
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time,"- The are",str(query_species_num),"species in the proteome folder.")

    # for development
    for species_i in range(query_species_num):
        len_prot_record_i = len( prots_record_allspecies[species_i] )
        species_name_i = query_species_names[species_i]
        #print(species_name_i,len_prot_record_i)
        if species_name_i in list_oma_speices: 
            current_time = datetime.now().strftime("%H:%M:%S")
            print(current_time,"- the species",species_name_i," already exists in the oma database, remove them first")
            exit()

      
    # The proteins are parsed using  Bio.SeqIO.parse
    # the first part of the header line before space 
    # >tr|A0A2I3FYY2|A0A2I3FYY2_NOMLE Uncharacterized protein OS=Nomascus leucogenys OX=61853 GN=CLPTM1L PE=3 SV=1
    # will be ">tr|A0A2I3FYY2|A0A2I3FYY2_NOMLE"
    # [i.id for i in prots_record_allspecies[0] if len(i.id)!=30 and len(i.id)!=22 ] #'sp|O47892|CYB_NOMLE',
    

    
    return (query_species_names, prots_record_allspecies)


def parse_hogmap_omamer(query_species_names):

    ################### Parsing omamer's output  ########
    #####################################################
    
    prots_hogmap_name_allspecies = []
    prots_hogmap_hogid_allspecies = []
    prots_hogmap_subfscore_allspecies = []
    prots_hogmap_seqlen_allspecies = []
    prots_hogmap_subfmedseqlen_allspecies = []

    for query_species_name in query_species_names:
        omamer_output_address = address_working_folder + "omamer_search/hogmap/"+ query_species_name + ".hogmap"     
        omamer_output_file = open(omamer_output_address,'r');
        prots_hogmap_name = []
        prots_hogmap_hogid = []
        prots_hogmap_subfscore = []
        prots_hogmap_seqlen = []
        prots_hogmap_subfmedseqlen = []
        
        for line in omamer_output_file:
            line_strip=line.strip()
            if not line_strip.startswith('qs'):
                line_split= line_strip.split("\t")    
                #if line_split[1]!='na':
                prots_hogmap_name.append(line_split[0])
                prots_hogmap_hogid.append(line_split[1])
                prots_hogmap_subfscore.append(line_split[4]) # subfamily
                prots_hogmap_seqlen.append(line_split[5])
                prots_hogmap_subfmedseqlen.append(line_split[6])
                
        prots_hogmap_name_allspecies.append(prots_hogmap_name)
        prots_hogmap_hogid_allspecies.append(prots_hogmap_hogid)
        prots_hogmap_subfscore_allspecies.append(prots_hogmap_subfscore)
        prots_hogmap_seqlen_allspecies.append(prots_hogmap_seqlen)
        prots_hogmap_subfmedseqlen_allspecies.append(prots_hogmap_subfmedseqlen)
    
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time,"- There are ",len(prots_hogmap_name_allspecies)," species in the hogmap folder.")
    print(current_time,"- The first species",query_species_names[0]," contains ",len(prots_hogmap_hogid_allspecies[0])," proteins.")
    print(current_time,"- The first protein of first species is ", prots_hogmap_name_allspecies[0][0])

    hogmap_allspecies = (prots_hogmap_name_allspecies, prots_hogmap_hogid_allspecies, prots_hogmap_subfscore_allspecies, prots_hogmap_seqlen_allspecies, prots_hogmap_subfmedseqlen_allspecies)
    return  hogmap_allspecies
    
    
    
def filter_prot_mapped(query_species_names, query_prot_records_species,query_prot_names_species_mapped):
    # omamer remove very small proteins, 
    # so  we lose track of order comparing hogmap and fasta file
    # the goal here is to remove those from seq record (of the fasta file)
    current_time = datetime.now().strftime("%H:%M:%S")
    print(current_time,"- Filtering proteins started.")

    query_prot_records_species_filtered=[]
    for species_idx in range(len(query_species_names)):    
        # from fasta file
        query_species_name=query_species_names[species_idx]
        print(query_species_name)
        query_prot_records_species_i = query_prot_records_species[species_idx]
        query_prot_ids_records = [record.id for record in query_prot_records_species_i]

        # from hogmap file
        # without proteins that are not mapped on any hogs
        query_prot_names_species_i = query_prot_names_species_mapped[species_idx]

        if len(query_prot_names_species_i) != len(query_prot_records_species_i):

            query_prot_records_filterd=[]
            for query_prot_name in query_prot_names_species_i:
                if query_prot_name in query_prot_ids_records:
                    prot_record_idx = query_prot_ids_records.index(query_prot_name)
                    prot_record = query_prot_records_species_i[prot_record_idx]
                    query_prot_records_filterd.append(prot_record)
                else:
                    current_time = datetime.now().strftime("%H:%M:%S")                    
                    print(current_time,"- Error",query_species_name, query_prot_name)

            current_time = datetime.now().strftime("%H:%M:%S")        
            print(current_time,"- For the species", query_species_name, ", few proteins were ignored by omamer.")
            print(current_time,"- before filtering: in hogmap", len(query_prot_names_species_i), "in proteome", len(query_prot_records_species_i))
            print(current_time,"- After filtering:  in hogmap", len(query_prot_names_species_i), "in proteome", len(query_prot_records_filterd))            
            

        else:
            query_prot_records_filterd = query_prot_records_species_i

        query_prot_records_species_filtered.append(query_prot_records_filterd)
    current_time = datetime.now().strftime("%H:%M:%S")        
    print(current_time,"- For the rest of species, all proteins were mapped using OMAmer.")

    return query_prot_records_species_filtered


def run_one_msa(seqRecords_queries):
    ############## MSA  ##############
    ##################################
    #current_time = datetime.now().strftime("%H:%M:%S")
    #print(current_time, "- working on new OG with length of ",len(seqRecords_OG_queries))
    
    
    wrapper_mafft = mafft.Mafft(seqRecords_queries,datatype="PROTEIN") 
    # MAfft error: Alphabet 'U' is unknown. -> add --anysymbol argument needed to define in the sourse code
    # workaround sed "s/U/X/g"
    
    wrapper_mafft.options.options['--retree'].set_value(1)


    run_mafft = wrapper_mafft() # it's wrapper  storing the result  and time 
    time_taken_mafft = wrapper_mafft.elapsed_time

    result_mafft = wrapper_mafft.result 
    time_taken_mafft2 = wrapper_mafft.elapsed_time
    
    current_time = datetime.now().strftime("%H:%M:%S")
    #print(current_time,"- time elapsed for MSA: ",time_taken_mafft2)
    #print(current_time,"- MSA for an OG is just finished: ",time_taken_mafft2)

    return(result_mafft)





def draw_tree(msa, tree_out_file):
    ############## Tree inference  ###################
    ##################################################

    wrapper_tree=fasttree.Fasttree(msa,datatype="PROTEIN")
    wrapper_tree.options.options['-fastest']    
    result_tree1 = wrapper_tree()

    time_taken_tree = wrapper_tree.elapsed_time 
    time_taken_tree

    result_tree2 = wrapper_tree.result
    tree_nwk=str(result_tree2["tree"])
    current_time = datetime.now().strftime("%H:%M:%S")
    #print(current_time,"- ",len(tree_nwk))

    if len(tree_out_file)>255: tree_out_file = tree_out_file[:255]
    file1 = open(tree_out_file,"w")
    file1.write(tree_nwk)
    file1.write(";\n")
    file1.close() 
    return tree_nwk



def merge_msa(list_msas):
    
    #logging.debug(list_msas)
    #logging.debug(str(list_msas[0][0].seq)+"\n")
    #logging.debug(str(list_msas[0][0].id)+"\n")
    #logging.debug(str(list_msas[1][0].seq)+"\n")
    #logging.debug(str(list_msas[0][0].id)+"\n")
    
    # each element of msa should be  a MultipleSeqAlignment
    wrapper_mafft_merge = mafft.Mafft(list_msas, datatype="PROTEIN") 
    wrapper_mafft_merge.options['--merge'].active = True
    merged = wrapper_mafft_merge()
    
    
    print(len(list_msas),"msas are merged into one with the length of ",len(merged),len(merged[0]) )
    return merged




    
    
# def compact_distance_matrix_tree(tree_input):

#     # tree_input   in ete3 format
#     # output  a vector upper triangulare 
    
#     tree_leaves=[]
#     for node in tree_input.traverse(strategy="postorder"):
#         if node.is_leaf() : 
#             node_name = node.name
#             tree_leaves.append(node_name)


#     leaves_num = len(tree_leaves)
#     distance_matrix = np.zeros([leaves_num,leaves_num])

#     for i in range(leaves_num):
#         for j in range(leaves_num):
#             if i < j:
#                 value= round(tree_input.get_distance(tree_leaves[i],tree_leaves[j]),3)
#                 distance_matrix[i][j]= value
#                 distance_matrix[j][i]= value

#     y=[]
#     for i in range(len(distance_matrix)):
#         for j in range(len(distance_matrix)):
#             if i<j:
#                 val= distance_matrix[i][j]
#                 y.append(val)
#     return (y,tree_leaves)

 
    
    
def lable_SD_internal_nodes(tree_out):

    species_name_dic={}
    counter_S=0
    counter_D=0
    
    for node in tree_out.traverse(strategy = "postorder"):
        #print("** now working on node ",node.name) # node_children

        if node.is_leaf() :
            prot_i = node.name
            species_name_dic[node] = { str(prot_i).split("|")[-1].split("_")[-1] }

        else:
            node.name= "S/D"
            leaves_list = node.get_leaves()
            #print("leaves_list", leaves_list)
            

            species_name_set = set([ str(prot_i).split("|")[-1].split("_")[-1] for prot_i in leaves_list])
            #print("species_name_set", species_name_set)
            species_name_dic[node] = species_name_set


            node_children = node.children
            #print(node_children)
            node_children_species_list = [species_name_dic[node_child] for node_child in node_children] # list of sets
            
            #print("node_children_species_list", node_children_species_list)
            
            node_children_species_intersection = set.intersection(*node_children_species_list)

            if  node_children_species_intersection :
                #print("node_children_species_list",node_children_species_list)
                counter_D += 1
                node.name = "D"+str(counter_D)
                
            else:
                counter_S += 1
                node.name = "S"+str(counter_S)

    return tree_out

def add_species_name(query_prot_records_species,query_species_names):

    for ix in range(len(query_species_names)):
        query_species_name = query_species_names[ix]
        query_prot_records = query_prot_records_species[ix]
        for i_prot in range(len(query_prot_records)):
            query_prot_record = query_prot_records[i_prot]
            query_prot_record.description += "|species|"+query_species_name
            
    return query_prot_records_species




def read_omaID_file(omaID_address):
    
    omaID_file = open(omaID_address,'r')

    taxonID_omaID={}
    omaID_taxonID={}

    # omaID_scienceFull={}
    # scienceFull_omaID={}
    scientific_taxonID={}
    taxonID_scientific={}

    #  !!! limitation  ignoring strains and isolate
    for line in omaID_file:
        line_strip = line.strip()
        if line_strip.startswith('#'):
            pass
            #header_lines_list.append(line_strip)
        else:
            line_parts = line_strip.split('\t')

            omaID = line_parts[0]
            taxonID = line_parts[1]
            taxonID_omaID[taxonID] = omaID
            omaID_taxonID[omaID] = taxonID

            scientific = line_parts[2]
            taxonID_scientific[taxonID] = scientific
            scientific_taxonID[scientific] = taxonID

    omaID_file.close()

    print("-- The map for OMA taxonID of",len(taxonID_omaID),"records have read.") 
    
    return (taxonID_omaID,omaID_taxonID,taxonID_scientific,scientific_taxonID)





def write_rootHOGs(prots_hogmap_hogid_allspecies, address_out_hog):

    prots_hogmap_rhogid_allspecies = []
    for prots_hogmap_hogid in prots_hogmap_hogid_allspecies:
        prots_hogmap_rhogid = []
        for prot_hogmap_hogid in prots_hogmap_hogid:
            prot_hogmap_rhogid=prot_hogmap_hogid.split(".")[0]
            prots_hogmap_rhogid.append(prot_hogmap_rhogid)

        prots_hogmap_rhogid_allspecies.append(prots_hogmap_rhogid)


    rhogid_prot_idx_dic = {}
    for species_idx in range(len(query_species_names)):

        species_name = query_species_names[species_idx]

        prots_hogmap_rhogid = prots_hogmap_rhogid_allspecies[species_idx]

        for prots_hogmap_idx in range(len(prots_hogmap_rhogid)):

            prot_hogmap_rhogid = prots_hogmap_rhogid[prots_hogmap_idx]
            if prot_hogmap_rhogid in rhogid_prot_idx_dic:
                rhogid_prot_idx_dic[prot_hogmap_rhogid].append((species_idx, prots_hogmap_idx))
            else:
                rhogid_prot_idx_dic[prot_hogmap_rhogid] = [(species_idx, prots_hogmap_idx)]
    print(len(rhogid_prot_idx_dic)) #  rhogid_prot_idx_dic['HOG:0018405']



    rhogids_prot_records_query = [ ]
    rhogids_list = []
    for rhogid in rhogid_prot_idx_dic.keys() :
        rhogid_prot_records = []
        if rhogid != "na" and len(rhogid)>1:
            rhogids_list.append(rhogid)
            rhogid_prot_idx =  rhogid_prot_idx_dic[rhogid]
            for (species_idx, prots_hogmap_idx) in rhogid_prot_idx:
                prot_record = query_prot_records_species_filtered[species_idx][prots_hogmap_idx] 
                #print(prot_record)
                rhogid_prot_records.append(prot_record)

            rhogids_prot_records_query.append(rhogid_prot_records) 
        else:
            print("root hog na / lenght of one ",rhogid)
    print(len(rhogids_prot_records_query),len(rhogids_prot_records_query[0]))
    
    
    #rhogids_prot_records = []
    rhogid_num_list= []
    for rhogid_idx in range(len(rhogids_list)):
        #if rhogid_idx %500 ==0 : print(rhogid_idx)
        rhogid_prot_records_query= rhogids_prot_records_query[rhogid_idx] 

        rhogid = rhogids_list[rhogid_idx]

        rhogid_B= rhogid.split(":")[1]
        rhogid_num= int(rhogid_B[1:] ) # # B0613860
        rhogid_num_list.append(rhogid_num)
        if  len(rhogid_prot_records_query) < 100  and len(rhogid_prot_records_query) > 2 :
    #         rhogids_prot_records_oma = []
    #         for hog_elements in oma_db.member_of_fam(rhogid_num):   # this gets the member of roothog 2 (HOG:000002)
    #             prot_hog_element = ProteinEntry(oma_db, hog_elements)
    #             #print(prot_hog_element.omaid, prot_hog_element.hog_family_nr, len(prot_hog_element.sequence),prot_hog_element.sequence[0])
    #             rhogids_prot_records_oma.append(SeqRecord(Seq(prot_hog_element.sequence), id=prot_hog_element.omaid))
    #         rhogids_prot_records_both= rhogids_prot_records_oma +  rhogid_prot_records_query
    #         rhogids_prot_records.append(rhogids_prot_records_both)
            SeqIO.write(rhogid_prot_records_query, address_out_hog+"HOG_"+str(rhogid_num)+".fa", "fasta")

    print("all HOGs   (>2 <100) has written.",len(rhogids_prot_records_query),len(rhogids_list), len(rhogid_prot_records_query), len(rhogid_prot_records_query[0]))

    return (rhogid_num_list, rhogids_prot_records_query)
    
    
def read_species_tree(tree_address):
    print(tree_address)
    #print(round(os.path.getsize(tree_address)/1000),"kb")

    project = Phyloxml()
    project.build_from_file(tree_address)
    # Each tree contains the same methods as a PhyloTree object
    for species_tree in project.get_phylogeny():
        len(species_tree)
        #print(species_tree)

    for node_species_tree in species_tree.traverse(strategy = "postorder"):
        if node_species_tree.is_leaf():
            temp1 =node_species_tree.phyloxml_clade.get_taxonomy()[0]
            #print(temp1.get_code())     
            node_species_tree.name = temp1.get_code()
    #print(len(species_tree))
    #print(species_tree)
    
    return (species_tree)   
    

    
def traverse_geneTree_assign_hog(gene_tree, merged_msa):
    # gene_tree should be labeled with S/ D
    tree_leaves = [ i.name for i in gene_tree.get_leaves()] 
    assigned_leaves_to_hog = []
    sub_msas_list_this_level = []

    for node in gene_tree.traverse(strategy = "preorder"): # start from root
        #print("Leaves assigned to hog are ", assigned_leaves_to_hog)
        #print("Traversing gene tree. Now at node", node.name)

        if node.is_root() and node.name[0] == "S":    
            sub_msas_list_this_level = [merged_msa]
            assigned_leaves_to_hog = tree_leaves
            # we do not need to traverse the gene tree any more 
            break

        if not node.is_leaf() : 
            node_leaves_name = [ i.name for i in node.get_leaves() ] 
            
            # ?? ?? 
            if node_leaves_name[0] in assigned_leaves_to_hog:
                # if one of them is there, since  preorder,  all should be there ???    
                # the node are assigned to hogs, we are done. 
                # it is not needed to check the children of a speciecation
                continue  # go to next node

            if node.name[0] =="S":
                # this is a sub-hog.
                assigned_leaves_to_hog += node_leaves_name
                leaves_msa = [i.id for i in merged_msa]
                idx_species_list = [leaves_msa.index(i) for i in node_leaves_name] 
                #print("idx_species_list",idx_species_list)
                #print("node_leaves_name",node_leaves_name)
                
                sub_msa_seq_list = [  merged_msa[i]   for i in idx_species_list]  # sub_msas_list_lowerlevel 
                sub_msa = run_one_msa(sub_msa_seq_list)
                sub_msas_list_this_level.append(sub_msa)

                if  set(tree_leaves)==set(assigned_leaves_to_hog) :
                    # all leaves are assigned to hogs, we are done. 
                    break

        if node.is_leaf() and not node.name in assigned_leaves_to_hog:
            #  singletone leaf, can be a hog. 
            assigned_leaves_to_hog.append(node.name)
            #print("assigned_leaves_to_hog",assigned_leaves_to_hog)
            leaves_msa = [i.id for i in merged_msa]
            #print("leaves_msa",leaves_msa)

            idx_species = leaves_msa.index(node.name)
            sub_msa = MultipleSeqAlignment([merged_msa[idx_species]])
            sub_msas_list_this_level.append(sub_msa)
            
    return (assigned_leaves_to_hog, sub_msas_list_this_level)
            

def prepare_species_tree(rhog_i, species_tree):

    species_names_rhog= []
    prot_names_rhog=[]
    for rec in rhog_i:
        prot_name= rec.name # # 'tr|E3JPS4|E3JPS4_PUCGT
        #prot_name = prot_name_full.split("|")[1].strip() # # 'tr|E3JPS4|E3JPS4_PUCGT
        species_name = prot_name.split("|")[-1].split("_")[-1]
        if species_name=='RAT': species_name="RATNO"

        species_names_rhog.append(species_name)
        prot_names_rhog.append(prot_name)

    species_names_uniqe = set(species_names_rhog)
    print("number of unique species in the rHOG", len(species_names_uniqe))

    
    species_tree.prune(species_names_uniqe, preserve_branch_length=True )
    species_tree.write()

    for node in species_tree.traverse(strategy = "postorder"):
        node_name = node.name
        if len(node_name) <1: 
            if node.is_leaf():
                node.name = "leaf_"+str(num_leaves_no_name)
            else:
                node_children = node.children
                list_children_names = [node_child.name for node_child in node_children]
                node.name = '_'.join(list_children_names)

    print("Working on the following species tree.")
    #print(species_tree)
    
    return (species_tree, species_names_rhog, prot_names_rhog)



def find_paralog_thisLevel(tree_leaves, subHOG_thisLevel):
    
    paralog_set_thisLevel = set()
    for i in range(len(tree_leaves)):
        prot_i = tree_leaves[i]
        species_i = prot_i.split("|")[-1].split("_")[-1]
        for j in range(i+1):
            prot_j = tree_leaves[j]
            species_j = prot_j.split("|")[-1].split("_")[-1]
            if species_i == species_j:
                paralog_set_thisLevel.add((prot_i,prot_j))
                paralog_set_thisLevel.add((prot_j,prot_i))

    for i in range(len(subHOG_thisLevel)):
        subHOG_i = subHOG_thisLevel[i]
        for j in range(i):
            subHOG_j = subHOG_thisLevel[j]
            for i1 in range(len(subHOG_i)):
                prot_i = subHOG_i[i1]
                for j1 in range(len(subHOG_j)):
                    prot_j = subHOG_j[j1]
                    paralog_set_thisLevel.add((prot_i,prot_j))
                    paralog_set_thisLevel.add((prot_j,prot_i))
 
    return paralog_set_thisLevel



def infer_HOG_thisLevel(node_species_tree, rhog_i, species_names_rhog, dic_sub_msas, rhogid_num):

    sub_msa_list_lowerLevel = [] # including subHOGS of lower level 
    for node_child in node_species_tree.children:
        if  node_child.is_leaf():
            node_species_name = node_child.name
            # for a extant species 
            # extracting prot sequeincg of the species from the rootHOG
            interest_list = [idx  for idx in range(len(species_names_rhog)) if species_names_rhog[idx] == node_species_name ]
            rhog_part = [rhog_i[i] for i in interest_list]
            sub_msa = [MultipleSeqAlignment([i]) for i in rhog_part] 
        else:   # the child node is an internal node, subHOGs are inferred till now in traversing.
            print("sub msa for internal node", node_child.name,"is read from dic.")
            if node_child.name in dic_sub_msas:
                sub_msa  = dic_sub_msas[node_child.name]
            else:
                print("error 131, no sub msa for the internal node ",node_child.name, node_child)
                assert 2==1 
        sub_msa_list_lowerLevel += sub_msa


    if len(sub_msa_list_lowerLevel) <2:
        print("**** issue **  ", len(sub_msa_list_lowerLevel),sub_msa_list_lowerLevel)
        return (-1,-1,-1)
    
    print("There are",len(sub_msa_list_lowerLevel)," subHOGs in the lower level.")
    #print("We want to infer subHOGs at this level,i.e. merge few of them.")    
    #time.sleep(1)     # shall I wait to have all  msa run finsihed? 
    
    #all hog of child
    #get msa
    
    merged_msa = merge_msa(sub_msa_list_lowerLevel) 

    print("All subHOGs are merged, merged msa is with length of",len(merged_msa), len(merged_msa[0]),".")

    gene_tree_file =  address_working_folder + "/gene_trees_test/tree_"+str(rhogid_num)+"_"+str(node_species_tree.name)+".nwk"
    
    gene_tree_raw = draw_tree(merged_msa, gene_tree_file)
    gene_tree = Tree(gene_tree_raw+";", format=0)
    print("Gene tree is infered with length of",len(gene_tree),".")

    #gene_tree_i +=1
    R = gene_tree.get_midpoint_outgroup()
    gene_tree.set_outgroup(R)
    #print("Midpoint rooting is done for gene tree.")

    gene_tree = lable_SD_internal_nodes(gene_tree)
    print("Overlap speciation is done for internal nodes of gene tree.")
    print(str(gene_tree.write(format=1))[:-1]+str(gene_tree.name)+":0;")
    #print(gene_tree)

    subHOG_thisLevel = []
    # for  each speciestion node
        #HOG([ sub hog  for in withing speciestion node], msa= merged_msa    )
    
    (assigned_leaves_to_hog, sub_msas_list_this_level) = traverse_geneTree_assign_hog(gene_tree, merged_msa)

    tree_leaves = [ i.name for i in gene_tree.get_leaves()] 
    if set(tree_leaves)-set(assigned_leaves_to_hog) :
        print("error 234",assigned_leaves_to_hog, sub_msas_list_this_level)

    dic_sub_msas[node_species_tree.name] = sub_msas_list_this_level 
    print("Number of hog at the taxonomic level", node_species_tree.name ,"is ",len(sub_msas_list_this_level),". The number of proteins per hog is", [len(i) for i in  sub_msas_list_this_level])    

    # compare with  subHOG_lowerLevel merge if needed two are from different 
    subHOG_thisLevel =[]
    for submsa in sub_msas_list_this_level :
        subHOG_thisLevel.append([seq.id for seq in submsa])
    #print("subHOGs are:", subHOG_thisLevel)

    paralog_set_thisLevel = find_paralog_thisLevel(tree_leaves, subHOG_thisLevel)

    return (subHOG_thisLevel, paralog_set_thisLevel, dic_sub_msas)



def write_ortholog_rhog(parlog_set_rhog, prot_names_rhog, address_orhto_pair_file):

    all_pairs= set()
    for i in range(len(prot_names_rhog)):
        for j in range(i+1):
            prot_i = prot_names_rhog[i]
            prot_j = prot_names_rhog[j]
            all_pairs.add((prot_i,prot_j))
         
    #print(all_pairs)
    #print("para", parlog_set_rhog)
    file_ortho = open(address_orhto_pair_file, "a")
    for pair in all_pairs:
        if pair not in parlog_set_rhog:
            prot_i_5letter= pair[0].split("|")[1].strip()
            prot_j_5letter= pair[1].split("|")[1].strip()
            #print("**ortho pair is**",prot_i_5letter,prot_j_5letter)
            file_ortho.write(prot_i_5letter+"\t"+prot_j_5letter+"\n")
            
    file_ortho.close() 
    
    print("Some ortho pairs are appended in", address_orhto_pair_file)

    return 1






def traverse_speciesTree_inferHOG(species_tree, rhog_i, species_names_rhog, rhogid_num, prot_names_rhog):

    subHOG_all = []
    dic_sub_msas = {}
    parlog_set_rhog = set()

    # finding hogs at each level of species tree (from leaves to root, bottom up)    
    for node_species_tree in species_tree.traverse(strategy = "postorder"):
        #dic_sub_hogs[node_species_tree.name] = []
        dic_sub_msas[node_species_tree.name] = []
        if node_species_tree.is_leaf() : 
            continue
        print("\n"+"*"*15+"\n","Finding hogs for the taxonomic level:", node_species_tree.name,"\n")

        (subHOG_thisLevel, paralog_set_thisLevel, dic_sub_msas)=  infer_HOG_thisLevel(node_species_tree, rhog_i, species_names_rhog, dic_sub_msas, rhogid_num)
        if subHOG_thisLevel == -1: continue
        
        print("subHOG_thisLevel",subHOG_thisLevel)
        subHOG_all.append(subHOG_thisLevel)
        
        parlog_set_rhog |= paralog_set_thisLevel

        
    write_ortho = write_ortholog_rhog(parlog_set_rhog, prot_names_rhog, address_orhto_pair_file)


    return (subHOG_all)





def run_inference(rhogid_num):

    print("\n"+"="*50+"\n","Working on root hog:", rhogid_num,"\n") 

    prot_address = address_out_hog+"HOG_"+str(rhogid_num)+".fa"
    rhog_i = list(SeqIO.parse(prot_address, "fasta")) 
    print("number of proteins in the rHOG", len(rhog_i))
    (species_tree) = read_species_tree(tree_address)
    (species_tree, species_names_rhog, prot_names_rhog) = prepare_species_tree(rhog_i, species_tree)

    if len(rhog_i) ==1:
        subHOG_all_rhog = [prot_names_rhog]
        ortholog_set_rhog = set()
        
    else:
        (subHOG_all_rhog) = traverse_speciesTree_inferHOG(species_tree, rhog_i, species_names_rhog, rhogid_num, prot_names_rhog)
    
    print("Working on root hog:", rhogid_num," is finished.\n"+"="*50+"\n") 

    return (subHOG_all_rhog ) 

        


#from scipy.cluster.hierarchy import dendrogram, linkage, ward, leaves_list, fcluster

import ete3
from ete3 import Tree
from ete3 import Phyloxml

import numpy as np
import concurrent.futures
#import ast
#import pickle
#import zoo
#zoo.__file__

import os


import zoo.wrappers.aligners.mafft as mafft  # mafft should be installed beforehand
import zoo.wrappers.treebuilders.fasttree as fasttree
import logging

import time
from datetime import datetime
from sys import argv
from os import listdir
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq # , UnknownSeqs
from Bio.Align import MultipleSeqAlignment

from collections import defaultdict

import matplotlib   #for development 
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

from random import sample


In [217]:
from pyoma.browser.models import ProteinEntry
import pyoma.browser.db as db
from pyoma.browser.hoghelper import build_hog_to_og_map


In [218]:



# if __name__ == "__main__":

#     print("program is started ")
#     address_working_folder = "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastget/qfo/"
#     address_out_hog= address_working_folder+ "hog_out_g2_s100/"
    
#     rHog_is_ready= False 
#     if not rHog_is_ready : 

if 1:
        oma_database_address = address_working_folder+"omamer_database/oma_path/OmaServer.h5"
        print("program has started. The oma database address is in ",oma_database_address)
        (oma_db, list_oma_speices) = parse_oma_db(oma_database_address)

        (query_species_names, query_prot_records_species) = parse_proteome(list_oma_speices)   
        query_prot_records_species = add_species_name(query_prot_records_species,query_species_names)

#         hogmap_allspecies = parse_hogmap_omamer(query_species_names)
#         (query_prot_names_species_mapped, prots_hogmap_hogid_allspecies, prots_hogmap_subfscore_allspecies, prots_hogmap_seqlen_allspecies, prots_hogmap_subfmedseqlen_allspecies) = hogmap_allspecies 
#         query_prot_records_species_filtered =  filter_prot_mapped(query_species_names, query_prot_records_species, query_prot_names_species_mapped)

#         print(len(query_prot_records_species_filtered),len(query_prot_records_species_filtered[0]))

#         ## Write  rootHoGs as fasta file
#         (rhogid_num_list, rhogids_prot_records_query) =  write_rootHOGs(prots_hogmap_hogid_allspecies, address_out_hog)
 

outdated database version, but only minor version change: 3.5 != 3.4. Some functions might fail


program has started. The oma database address is in  /work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastget/qfo/omamer_database/oma_path/OmaServer.h5


Cannot load SequenceSearch. Any future call to seq_search will fail!
Traceback (most recent call last):
  File "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/software/miniconda3/lib/python3.8/site-packages/pyoma/browser/db.py", line 2035, in __init__
    self.seq_idx = self.seq_idx()
  File "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/software/miniconda3/lib/python3.8/site-packages/tables/link.py", line 393, in __call__
    self.extfile = tables.open_file(filename, **kwargs)
  File "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/software/miniconda3/lib/python3.8/site-packages/pyoma/browser/db.py", line 113, in synchronized_open_file
    return _tables_file._original_open_file(*args, **kwargs)
  File "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/software/miniconda3/lib/python3.8/site-packages/tables/file.py", line 315, in open_file
    return File(filename, mode, title, root_uep, filters, **kwargs)
  File "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/software/miniconda3/lib/python3.8/site-pack

15:32:58 - OMA data is parsed and its release name is: All.Dec2021
15:32:58 - There are 2496 species in the OMA database.
15:33:17 - The are 78 species in the proteome folder.


In [219]:



# if __name__ == "__main__":

    
    
    
#     print("program is started ")
    
#     address_working_folder = "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastget/qfo/"
#     address_out_hog= address_working_folder+ "hog_out_g2_s100/"
    
#     rHog_is_ready= True 
#     if not rHog_is_ready : pass

    
    
    
#     tree_address = address_working_folder+"lineage_tree_qfo.phyloxml"
    
#     rhog_files = listdir(address_out_hog)
#     rhogid_num_list= []
#     for rhog_file in rhog_files:
#         if rhog_file.split(".")[-1] == "fa":
#             rhogid_num = int(rhog_file.split(".")[0].split("_")[1])
#             rhogid_num_list.append(rhogid_num)

    
#     try : 
#         rhogid_num_list2 = rhogid_num_list[1002:1110] #[159556, 105904] #rhogid_num_list[2000:2010]
#         dic_solved={}
#         list_done=[]
#         for i in rhogid_num_list2: dic_solved[i] = 0

#         subHOG_all= [] # list of list
#         # make sure the file is empty
#         address_orhto_pair_file =  address_working_folder+"ortho_pair_first.tsv"
#         file_ortho = open(address_orhto_pair_file, "w")
#         file_ortho.close()


#         print("parrallel is started")
#         number_max_workers = 20
#         with concurrent.futures.ProcessPoolExecutor(max_workers = number_max_workers) as executor: 
#             for rhogid_num, output_values in zip(rhogid_num_list2, executor.map(run_inference, rhogid_num_list2)):
#                 (subHOG_all_rhog) = output_values
#                 dic_solved[rhogid_num]=1
#                 list_done.append(rhogid_num)
                


#         print("all done !!")
#         print("program is finished ")
#     except:
#         print("program faced an error ",rhogid_num)        
#         print("all",rhogid_num_list2)
#         print("****** done", list_done)
#         index_last= rhogid_num_list2.index(list_done[-1])
#         print(index_last)
#         print(rhogid_num_list2[index_last-1:index_last+5])





In [278]:
def infer_HOG_thisLevel(node_species_tree, rhog_i, species_names_rhog, dic_sub_hogs, rhogid_num):

    sub_msa_list_lowerLevel = [] # including subHOGS of lower level 
    subHOGs_children = []

    print("working on node", node_species_tree.name,"with",len(node_species_tree.children),"children.")
    for node_child in node_species_tree.children:
        if  node_child.is_leaf():
            node_species_name = node_child.name
            #extracting those proteins of the rHOG that belongs to this species (child node of species tree)             
            interest_list = [idx  for idx in range(len(species_names_rhog)) if species_names_rhog[idx] == node_species_name ]
            rhog_part = [rhog_i[i] for i in interest_list]
            #sub_msa = [MultipleSeqAlignment([i]) for i in rhog_part] 
            #print("len",len(rhog_part))

            #sub_hog_leaf_list=[]
            for prot in rhog_part : 
                sub_hog_leaf = HOG(prot, node_species_name ) # node_species_tree.name
                #list_all_hogs_ever.append(sub_hog_leaf)
                subHOGs_children.append(sub_hog_leaf)                
                #sub_hog_leaf_list.append(sub_hog_leaf)
            #sub_msa = sub_hog_leaf_list

        else:   # the child node is an internal node, subHOGs are inferred till now during traversing.

            print("sub msa for internal node", node_child.name,"is read from dic.")
            #if node_child.name in dic_sub_msas:
                #sub_msa  = dic_sub_msas[node_child.name]
            if node_child.name in dic_sub_hogs:
                sub_hogs_child  = dic_sub_hogs[node_child.name]
                subHOGs_children += sub_hogs_child
            else:
                print("error 131, no sub msa for the internal node ",node_child.name, node_child)
                assert 2==1 
    temp11=[]
    for temp in [i._members for i in subHOGs_children]:
        temp11.append([ prot.split('|')[2] for prot in temp])

    print("there are ",len(subHOGs_children),"subHOGs lower of this level:",[i._hogid for i in subHOGs_children],temp11)
    print("We want to infer subHOGs at this level,i.e. merge few of them.")    

    if len(subHOGs_children) <2:
        print("**** error 134 *** ", len(subHOGs_children),subHOGs_children)
            #return (-1,-1,-1)
    else:

        sub_msa_list_lowerLevel_ready = [hog._msa for hog in subHOGs_children]
        merged_msa = merge_msa(sub_msa_list_lowerLevel_ready) 

        print("All subHOGs are merged, merged msa is with length of",len(merged_msa), len(merged_msa[0]),".")

        gene_tree_file =  address_working_folder + "/gene_trees_test/tree_"+str(rhogid_num)+"_"+str(node_species_tree.name)+".nwk"
        gene_tree_raw = draw_tree(merged_msa, gene_tree_file)
        gene_tree = Tree(gene_tree_raw+";", format=0)
        print("Gene tree is infered with length of",len(gene_tree),".")

        #gene_tree_i +=1
        R = gene_tree.get_midpoint_outgroup()
        gene_tree.set_outgroup(R)
        #print("Midpoint rooting is done for gene tree.")
        gene_tree = lable_SD_internal_nodes(gene_tree)
        print("Overlap speciation is done for internal nodes of gene tree, as following:")
        print(str(gene_tree.write(format=1))[:-1]+str(gene_tree.name)+":0;")
        #print(gene_tree)
        #(assigned_leaves_to_hog, sub_msas_list_this_level) = traverse_geneTree_assign_hog(gene_tree, merged_msa)

        tree_leaves = [i.name for i in gene_tree.get_leaves() ]
        #assigned_leaves_to_hog = []        #sub_msas_list_this_level = []
        subHOGs_id_children_assigned = [] # the same as  subHOG_to_be_merged_all_id 
        HOG_this_level = []
        subHOG_to_be_merged_set_other_Snodes = []
        subHOG_to_be_merged_set_other_Snodes_flattned_temp  = []
        for node in gene_tree.traverse(strategy = "preorder", is_leaf_fn= lambda n :   hasattr(n, "processed") and   n.processed == True  ): # start from root
            #print("Leaves assigned to hog are ", assigned_leaves_to_hog)   #print("Traversing gene tree. Now at node", node.name)
            if not node.is_leaf() : 
                node_leaves_name = [ i.name for i in node.get_leaves() ] 
                print(node_leaves_name)

                if node.name[0] =="S":
                    # this is a sub-hog.
                    subHOG_to_be_merged = [ ]
                    for node_leave_name in node_leaves_name:
                        #print(node_leave_name)
                        for subHOG in subHOGs_children :
                            subHOG_members= subHOG._members
                            if node_leave_name in subHOG_members:
                                
                                
                                # conflict ????? 
                        
                                # could be improved
                                if subHOG._hogid  not in subHOG_to_be_merged_set_other_Snodes_flattned_temp:

                                    subHOG_to_be_merged.append(subHOG)
                                    subHOGs_id_children_assigned.append(subHOG._hogid)
                                else:
                                    print("issue 184",node.name,subHOG._hogid, node_leave_name)
                                    if  "processed" in  node:
                                        print(node.name)
                                    else:
                                        print("processed not in ", node.name)
                                        
                                # print(node_leave_name,"is in ",subHOG._hogid)
                    if subHOG_to_be_merged :
                        subHOG_to_be_merged_set = set(subHOG_to_be_merged)     
                        taxnomic_range= node_species_tree.name
                        HOG_this_node = HOG(subHOG_to_be_merged_set,taxnomic_range , msa= merged_msa) 
                        HOG_this_level.append(HOG_this_node)
                        subHOG_to_be_merged_set_other_Snodes.append([i._hogid for i in subHOG_to_be_merged_set])
                        subHOG_to_be_merged_set_other_Snodes_flattned_temp= [item for items in subHOG_to_be_merged_set_other_Snodes for item in items]       
                    
                    
                    #  I don't need to traverse deeper in this clade
                    
                    node.processed = True
                    #print("?*?*  ", node.name)
                    

                    
            subHOG_to_be_merged_set_other_Snodes_flattned= [item for items in subHOG_to_be_merged_set_other_Snodes for item in items]       
            if  [i._hogid for i in subHOGs_children] == subHOG_to_be_merged_set_other_Snodes_flattned:
                break

        for subHOG in subHOGs_children :      
            # for the single branch  ( D include a  subhog and a S node. )
            if  subHOG._hogid  not in subHOGs_id_children_assigned : 
                #print("here", subHOG)
                HOG_this_level.append(subHOG)
        prot_list_sbuhog= [i._members for i in HOG_this_level]
        prot_list_sbuhog_short = []
        for prot_sub_list_sbuhog in prot_list_sbuhog:
            prot_list_sbuhog_short.append([ prot.split('|')[2] for prot in prot_sub_list_sbuhog])
            
        
        print("- ",len(prot_list_sbuhog_short),"HOGs are inferred at the level ",node_species_tree.name,":",prot_list_sbuhog_short )
    print("By merging ",subHOG_to_be_merged_set_other_Snodes)

        #check for conflic in merging
    #     for i in range(subHOG_to_be_merged_set_other_Snodes):
    #         if 
    #         for i in range(subHOG_to_be_merged_set_other_Snodes):
    dic_sub_hogs[node_species_tree.name] = HOG_this_level
    return (dic_sub_hogs)


In [279]:
a=2
a

2

In [280]:

address_working_folder = "/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastget/qfo/"
address_out_hog= address_working_folder+ "hog_out_g2_s100/"

rHog_is_ready= True 
if not rHog_is_ready : pass

tree_address = address_working_folder+"lineage_tree_qfo.phyloxml"

rhog_files = listdir(address_out_hog)
rhogid_num_list= []
for rhog_file in rhog_files:
    if rhog_file.split(".")[-1] == "fa":
        rhogid_num = int(rhog_file.split(".")[0].split("_")[1])
        rhogid_num_list.append(rhogid_num)

        
        


subHOG_all= [] # list of list

address_orhto_pair_file =  address_working_folder+"ortho_pair_first_test.tsv"
file_ortho = open(address_orhto_pair_file, "w")
file_ortho.close()

In [281]:
import itertools

In [282]:
class HOG:
    
    _hogid_iter = 100 
    def __init__(self, input_instantiate, taxnomic_range,  msa = None):       # _prot_names
        
        # the input_instantiate could be either
        #     1) a protein as the biopython seq record  SeqRecord(seq=Seq('MAPSSRSPSPRT. ] 
        # or  2) a set of intances of class HOG   wit a big msa
        
        # those variable starting with _ are local to the class, should not access directly  (although it is possbile)
        self.__class__._hogid_iter += 1
        self._hogid= "hog"+str(self.__class__._hogid_iter)
        self._taxnomic_range = taxnomic_range
        #print("**** a new HOG is instantiated with id", self._hogid)
        
        if  isinstance(input_instantiate, SeqRecord):    #if len(sub_hogs)==1:
            only_protein = input_instantiate
            # only one seq, only on child, leaf            
            self._members = set([only_protein.id])
            self._msa =  MultipleSeqAlignment([only_protein])
            self._subhogs = []
            # <<class 'Bio.Align.MultipleSeqAlignment'> instance (1 records of length 314) at 7f0e86c713d0>
            
        elif     msa    and   all(isinstance(x, HOG) for x in input_instantiate):  
            # here we want to merge few subHOGs and creat a new HOG.
            # the n
            
            sub_hogs = input_instantiate
            
            hog_members = set()
            for sub_hog in sub_hogs: 
                hog_members |= sub_hog.get_members()  #union
            self._members =  hog_members              #set.union(*tup) 
            # full members 
            self._subhogs = list(input_instantiate)
            
           
            
            # subsampling in msa 
            max_num_seq=30
            
            records_full = [record for record in msa if record.id in self._members]
            if len(records_full)> max_num_seq: 
                records_sub_sampled = sample(records_full, max_num_seq)   #  without replacement.
                print("we are doing subsamping now from",len(records_full), "to",max_num_seq,"seq")
            else:
                records_sub_sampled = records_full
            
            # removing some columns completely gap -  (not x   )
            
            # now select those proteins 
            self._msa =  MultipleSeqAlignment(records_sub_sampled)
            # without replacement sampling , 
            # self._children = sub_hogs # as legacy  ?
            
            
        else:
            print("Error 136,  check the input format to instantiate a HOG class")
            assert False

    def __repr__(self):
        
        return "class HOG with lenght"+str(len(self._members))+"taxonimic range"+str(self._taxnomic_range)
            
            
    def get_members(self):
        return set(self._members)
        
        #merge, gene tree, midpoint, lable_SD_internal_nodes, traverse_geneTree_assign_hog
#     def to_orthoxml(self):
#          if len(self._members) == 1:   
#             out = "xml.generef"
#             return out
#         else:
#             out=1

    def to_orthoxml(self, indent=0):
        if len(self._subhogs) == 0:
            return ET.Element('geneRef', attrib={'id': list(self._members)[0]})
        
        hog_elemnt = ET.Element('orthologGroup')
        def _sorter_key(sh): 
            return sh._taxnomic_range
        self._subhogs.sort(key=_sorter_key)
        print(f'{" "*indent}subhog: {self._taxnomic_range}:')
        for sub_clade, sub_hogs in itertools.groupby(self._subhogs, key=_sorter_key):
            list_of_subhogs_of_same_clade = list(sub_hogs)
            print(f'{" "*(indent+1)} clade: {sub_clade} with {str(len(list_of_subhogs_of_same_clade))}')
            if len(list_of_subhogs_of_same_clade) > 1:
                paralog_element = ET.Element('paralogGroup')
                for sh in list_of_subhogs_of_same_clade:
                    paralog_element.append(sh.to_orthoxml(indent+2))
                hog_elemnt.append(paralog_element)
            else:
                hog_elemnt.append(list_of_subhogs_of_same_clade[0].to_orthoxml(indent+2))
        return hog_elemnt


In [283]:


for rhogid_num in rhogid_num_list[39:40]:  # 39 169

    print("\n"+"="*50+"\n","Working on root hog:", rhogid_num,"\n") 

    prot_address = address_out_hog+"HOG_"+str(rhogid_num)+".fa"
    rhog_i = list(SeqIO.parse(prot_address, "fasta")) 
    print("number of proteins in the rHOG", len(rhog_i))
    (species_tree) = read_species_tree(tree_address)

    (species_tree, species_names_rhog, prot_names_rhog) = prepare_species_tree(rhog_i, species_tree)

    print("\n species_names_rhog ",set(species_names_rhog))

    #species_tree.write()
    print(species_tree.write())
    print(species_tree)


    #subHOG_all = []
    dic_sub_hogs = {}
    #parlog_set_rhog = set()

    # finding hogs at each level of species tree (from leaves to root, bottom up)    
    for node_species_tree in species_tree.traverse(strategy = "postorder"):
        #dic_sub_hogs[node_species_tree.name] = []
        #dic_sub_msas[node_species_tree.name] = []
        if node_species_tree.is_leaf() : 
            continue
        print("\n"+"*"*15+"\n","Finding hogs for the taxonomic level:", node_species_tree.name,"\n")
        dic_sub_msas=[]
        (dic_sub_hogs) = infer_HOG_thisLevel(node_species_tree, rhog_i, species_names_rhog, dic_sub_hogs, rhogid_num)
        #print("&& ** %%",dic_sub_hogs)
        #(subHOG_thisLevel, paralog_set_thisLevel, dic_sub_msas)=  infer_HOG_thisLevel(node_species_tree, rhog_i, species_names_rhog, dic_sub_msas, rhogid_num)


    #write_ortho = write_ortholog_rhog(parlog_set_rhog, prot_names_rhog, address_orhto_pair_file)




 Working on root hog: 569382 

number of proteins in the rHOG 14
/work/FAC/FBM/DBC/cdessim2/default/smajidi1/fastget/qfo/lineage_tree_qfo.phyloxml
number of unique species in the rHOG 14
Working on the following species tree.

 species_names_rhog  {'GORGO', 'CHICK', 'CANLF', 'XENTR', 'BOVIN', 'LEPOC', 'ORYLA', 'MONDO', 'MOUSE', 'NEMVE', 'PANTR', 'DANRE', 'HUMAN', 'RATNO'}
(NEMVE:0,((LEPOC:0,(ORYLA:0,DANRE:0)1:0)1:0,((((((MOUSE:0,RATNO:0)1:0,(GORGO:0,HUMAN:0,PANTR:0)1:0)1:0,(CANLF:0,BOVIN:0)1:0)1:0,MONDO:0)1:0,CHICK:0)1:0,XENTR:0)1:0)1:0);

   /-NEMVE
  |
  |      /-LEPOC
  |   /-|
  |  |  |   /-ORYLA
  |  |   \-|
--|  |      \-DANRE
  |  |
  |  |                  /-MOUSE
  |  |               /-|
  |  |              |   \-RATNO
  |  |            /-|
  |  |           |  |   /-GORGO
   \-|           |  |  |
     |           |   \-|--HUMAN
     |         /-|     |
     |        |  |      \-PANTR
     |        |  |
     |      /-|  |   /-CANLF
     |     |  |   \-|
     |     |  |      \-B

In [284]:
a= dic_sub_hogs["LEPOC_ORYLA_DANRE_MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK_XENTR"][0]

In [285]:
res = a.to_orthoxml()
rough_string = ET.tostring(res, 'utf-8')
reparsed = minidom.parseString(rough_string)
print(reparsed.toprettyxml(indent="  "))

subhog: LEPOC_ORYLA_DANRE_MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK_XENTR:
  clade: LEPOC_ORYLA_DANRE with 1
  subhog: LEPOC_ORYLA_DANRE:
    clade: LEPOC with 1
    clade: ORYLA_DANRE with 1
    subhog: ORYLA_DANRE:
      clade: DANRE with 1
      clade: ORYLA with 1
  clade: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK_XENTR with 1
  subhog: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK_XENTR:
    clade: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK with 1
    subhog: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK:
      clade: CHICK with 1
      clade: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO with 1
      subhog: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO:
        clade: MONDO with 1
        clade: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN with 1
        subhog: MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN:
          clade: CANLF_BOVIN with 1
          subhog: CANLF_BOVIN:
            clade: BOVIN with 1
            clade: CANLF w

In [ ]:
# (query_species_names, query_prot_records_species) = parse_proteome(list_oma_speices)   
# query_prot_records_species = add_species_name(query_prot_records_species,query_species_names)


In [186]:
query_prot_record

SeqRecord(seq=Seq('MHRRARRMPMRPRRSKRVRNRYTMGTFALHGLTHRLPSASLQTTAARHPDVTQF...HYR'), id='sp|E2FZM4|SOCA_MYCTU', name='sp|E2FZM4|SOCA_MYCTU', description='sp|E2FZM4|SOCA_MYCTU Uncharacterized protein SocA OS=Mycobacterium tuberculosis (strain ATCC 25618 / H37Rv) OX=83332 GN=socA PE=2 SV=1|species|UP000001584_83332', dbxrefs=[])

In [189]:

import xml.etree.ElementTree as ET

ortho_file  = ET.Element("orthoXML") # xmlns=\"http://orthoXML.org/2011/\"
database    = ET.SubElement(ortho_file, "database", attrib={"name":"human", "NCBITaxId" :"63"}) # 


gene_counter =0
gene_id_name = {}

for species_i in range(len(query_species_names[12:15])):
    species_name = query_species_names[species_i]
    species = ET.SubElement(database, "species", attrib={"name":species_name, "NCBITaxId":"1"})
    query_prot_records =  query_prot_records_species[species_i]
    for gene_i in range(len(query_prot_records[12:15])):
        query_prot_record= query_prot_records[gene_i]
        
        gene = ET.SubElement(species, "gene", attrib={"id":str(gene_counter), "protId":query_prot_record.id})
        
        gene_id_name[query_prot_record.id]= gene_counter
        
        gene_counter += 1

#species.protId
        
ET.dump(ortho_file)  

#ET.tostring(ortho_file, xml_declaration=True, encoding='utf8').decode()
et = ET.ElementTree(ortho_file)
et.write("output1.xml")
# # .write('output.xml')
# !cat "output1.xml"

print("\n \n ")

rough_string = ET.tostring(ortho_file, 'utf-8')
reparsed = minidom.parseString(rough_string)
print(reparsed.toprettyxml(indent="  "))

<orthoXML><database name="human" NCBITaxId="63"><species name="UP000008783_418459" NCBITaxId="1"><gene id="0" protId="tr|E3JPS1|E3JPS1_PUCGT" /><gene id="1" protId="tr|E3JPS2|E3JPS2_PUCGT" /><gene id="2" protId="tr|E3JPS3|E3JPS3_PUCGT" /></species><species name="UP000002485_284812" NCBITaxId="1"><gene id="3" protId="sp|A0ZWU1|EAF1_SCHPO" /><gene id="4" protId="sp|A6X969|YI9B_SCHPO" /><gene id="5" protId="sp|A6X970|YF64_SCHPO" /></species><species name="UP000001584_83332" NCBITaxId="1"><gene id="6" protId="sp|A0A089QKZ7|Y155A_MYCTU" /><gene id="7" protId="sp|A0A089QRB9|MSL3_MYCTU" /><gene id="8" protId="sp|E2FZM4|SOCA_MYCTU" /></species></database></orthoXML>

 
 
<?xml version="1.0" ?>
<orthoXML>
  <database name="human" NCBITaxId="63">
    <species name="UP000008783_418459" NCBITaxId="1">
      <gene id="0" protId="tr|E3JPS1|E3JPS1_PUCGT"/>
      <gene id="1" protId="tr|E3JPS2|E3JPS2_PUCGT"/>
      <gene id="2" protId="tr|E3JPS3|E3JPS3_PUCGT"/>
    </species>
    <species name="UP0000

In [194]:
dic_sub_hogs["NEMVE_LEPOC_ORYLA_DANRE_MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK_XENTR"][0]._members

{'sp|A5PLL1|AN34B_HUMAN',
 'sp|Q3UUF8|AN34B_MOUSE',
 'tr|A0A1L1RSA7|A0A1L1RSA7_CHICK',
 'tr|A0A2I2ZNV7|A0A2I2ZNV7_GORGO',
 'tr|A0A2I3THT9|A0A2I3THT9_PANTR',
 'tr|A0A5F8H9I8|A0A5F8H9I8_MONDO',
 'tr|A0A6I8QDM6|A0A6I8QDM6_XENTR',
 'tr|A2BGQ0|A2BGQ0_DANRE',
 'tr|A7SYH3|A7SYH3_NEMVE',
 'tr|D3ZKP1|D3ZKP1_RAT',
 'tr|F1ME24|F1ME24_BOVIN',
 'tr|H2MS52|H2MS52_ORYLA',
 'tr|J9JHK4|J9JHK4_CANLF',
 'tr|W5NLW9|W5NLW9_LEPOC'}

In [196]:
dic_sub_hogs["NEMVE_LEPOC_ORYLA_DANRE_MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK_XENTR"]

# class object, __reper__

In [ ]:
dic_sub_hogs["NEMVE_LEPOC_ORYLA_DANRE_MOUSE_RATNO_GORGO_HUMAN_PANTR_CANLF_BOVIN_MONDO_CHICK_XENTR"]

In [233]:

import xml.etree.ElementTree as ET

ortho_file  = ET.Element("orthoXML") # xmlns=\"http://orthoXML.org/2011/\"
database    = ET.SubElement(ortho_file, "database", attrib={"name":"human", "NCBITaxId" :"63"}) # 

for species_i in range(len(query_species_names[12:15])):
    species_name = query_species_names[species_i]
    species = ET.Element("species", attrib={"name":species_name, "NCBITaxId":"1"})
    database.append(species)
    
    
    query_prot_records =  query_prot_records_species[species_i]
    for query_prot_record in query_prot_records[:10]:
        gene = ET.SubElement(species, "gene", attrib={"id":"1", "protId":query_prot_record.id})

#species.protId
        
ET.dump(ortho_file)  

#ET.tostring(ortho_file, xml_declaration=True, encoding='utf8').decode()
et = ET.ElementTree(ortho_file)
et.write("output1.xml")
# # .write('output.xml')
# !cat "output1.xml"


<orthoXML><database name="human" NCBITaxId="63"><species name="UP000008783_418459" NCBITaxId="1"><gene id="1" protId="tr|E3JPS1|E3JPS1_PUCGT" /><gene id="1" protId="tr|E3JPS2|E3JPS2_PUCGT" /><gene id="1" protId="tr|E3JPS3|E3JPS3_PUCGT" /><gene id="1" protId="tr|E3JPS4|E3JPS4_PUCGT" /><gene id="1" protId="tr|E3JPS5|E3JPS5_PUCGT" /><gene id="1" protId="tr|E3JPS7|E3JPS7_PUCGT" /><gene id="1" protId="tr|E3JPS8|E3JPS8_PUCGT" /><gene id="1" protId="tr|E3JPS9|E3JPS9_PUCGT" /><gene id="1" protId="tr|E3JPT0|E3JPT0_PUCGT" /><gene id="1" protId="tr|E3JPT1|E3JPT1_PUCGT" /></species><species name="UP000002485_284812" NCBITaxId="1"><gene id="1" protId="sp|A0ZWU1|EAF1_SCHPO" /><gene id="1" protId="sp|A6X969|YI9B_SCHPO" /><gene id="1" protId="sp|A6X970|YF64_SCHPO" /><gene id="1" protId="sp|A6X972|GON7_SCHPO" /><gene id="1" protId="sp|A6X974|YFQ8_SCHPO" /><gene id="1" protId="sp|A6X977|YEID_SCHPO" /><gene id="1" protId="sp|A6X978|YF12_SCHPO" /><gene id="1" protId="sp|A6X980|SFC7_SCHPO" /><gene id="1" p

In [234]:
rough_string = ET.tostring(ortho_file, 'utf-8')
reparsed = minidom.parseString(rough_string)
print(reparsed.toprettyxml(indent="  "))

<?xml version="1.0" ?>
<orthoXML>
  <database name="human" NCBITaxId="63">
    <species name="UP000008783_418459" NCBITaxId="1">
      <gene id="1" protId="tr|E3JPS1|E3JPS1_PUCGT"/>
      <gene id="1" protId="tr|E3JPS2|E3JPS2_PUCGT"/>
      <gene id="1" protId="tr|E3JPS3|E3JPS3_PUCGT"/>
      <gene id="1" protId="tr|E3JPS4|E3JPS4_PUCGT"/>
      <gene id="1" protId="tr|E3JPS5|E3JPS5_PUCGT"/>
      <gene id="1" protId="tr|E3JPS7|E3JPS7_PUCGT"/>
      <gene id="1" protId="tr|E3JPS8|E3JPS8_PUCGT"/>
      <gene id="1" protId="tr|E3JPS9|E3JPS9_PUCGT"/>
      <gene id="1" protId="tr|E3JPT0|E3JPT0_PUCGT"/>
      <gene id="1" protId="tr|E3JPT1|E3JPT1_PUCGT"/>
    </species>
    <species name="UP000002485_284812" NCBITaxId="1">
      <gene id="1" protId="sp|A0ZWU1|EAF1_SCHPO"/>
      <gene id="1" protId="sp|A6X969|YI9B_SCHPO"/>
      <gene id="1" protId="sp|A6X970|YF64_SCHPO"/>
      <gene id="1" protId="sp|A6X972|GON7_SCHPO"/>
      <gene id="1" protId="sp|A6X974|YFQ8_SCHPO"/>
      <gene id="1

In [177]:

import xml.etree.ElementTree as ET

#<groups>
#    <orthologGroup id="HOG:B0568884">
    
ortho_file  = ET.Element("orthoXML") # xmlns=\"http://orthoXML.org/2011/\"
database    = ET.SubElement(ortho_file, "database", attrib={"name":"human", "NCBITaxId" :"63"}) # 

groups = ET.SubElement(database, "groups")

orthologGroup = ET.SubElement(groups, "orthologGroup")# , attrib={"name":}
#<property name="TaxRange" value="Neopterygii"/>

geneRef = ET.SubElement(orthologGroup, "geneRef", attrib={"id":"8783511"}) # <geneRef id="8783511"/>
geneRef = ET.SubElement(orthologGroup, "geneRef", attrib={"id":"8783512"}) # <geneRef id="8783511"/>
orthologGroup = ET.SubElement(orthologGroup, "orthologGroup")# , attrib={"name":}
geneRef = ET.SubElement(orthologGroup, "geneRef", attrib={"id":"8783511"}) # <geneRef id="8783511"/>


orthologGroup = ET.SubElement(groups, "orthologGroup")# , attrib={"name":}
#<property name="TaxRange" value="Neopterygii"/>
geneRef = ET.SubElement(orthologGroup, "geneRef", attrib={"id":"22"}) # <geneRef id="8783511"/>
geneRef = ET.SubElement(orthologGroup, "geneRef", attrib={"id":"33"}) # <geneRef id="8783511"/>
geneRef = ET.SubElement(orthologGroup, "geneRef", attrib={"id":"44"}) # <geneRef id="8783511"/>

# orthologGroup = ET.SubElement(groups, "orthologGroup", attrib={"name":})

#property_ = ET.SubElement(orthologGroup, "property", attrib={"name":"TaxRange"})
#property

#     <orthologGroup id="HOG:B0568884">
#       <property name="TaxRange" value="Gnathostomata"/>


#for species_i in range(len(query_species_names[12:15])):
#     species_name = query_species_names[species_i]
#     species = ET.SubElement(database, "species", attrib={"name":species_name, "NCBITaxId":"1"})
#     query_prot_records =  query_prot_records_species[species_i]
#     for query_prot_record in query_prot_records[:10]:
#         gene = ET.SubElement(species, "gene", attrib={"id":"1", "protId":species_name})


# for species_i in range(len(query_species_names[12:15])):
#     species_name = query_species_names[species_i]
#     species = ET.SubElement(database, "species", attrib={"name":species_name, "NCBITaxId":"1"})
#     query_prot_records =  query_prot_records_species[species_i]
#     for query_prot_record in query_prot_records[:10]:
#         gene = ET.SubElement(species, "gene", attrib={"id":"1", "protId":species_name})

ET.dump(ortho_file)  

#ET.tostring(ortho_file, xml_declaration=True, encoding='utf8').decode()
et = ET.ElementTree(ortho_file)
#et.write("output1.xml")
# # .write('output.xml')
# !cat "output1.xml"


<orthoXML><database name="human" NCBITaxId="63"><groups><orthologGroup><geneRef id="8783511" /><geneRef id="8783512" /><orthologGroup><geneRef id="8783511" /></orthologGroup></orthologGroup><orthologGroup><geneRef id="22" /><geneRef id="33" /><geneRef id="44" /></orthologGroup></groups></database></orthoXML>


In [178]:


rough_string = ET.tostring(ortho_file, 'utf-8')
reparsed = minidom.parseString(rough_string)
print(reparsed.toprettyxml(indent="  "))

<?xml version="1.0" ?>
<orthoXML>
  <database name="human" NCBITaxId="63">
    <groups>
      <orthologGroup>
        <geneRef id="8783511"/>
        <geneRef id="8783512"/>
        <orthologGroup>
          <geneRef id="8783511"/>
        </orthologGroup>
      </orthologGroup>
      <orthologGroup>
        <geneRef id="22"/>
        <geneRef id="33"/>
        <geneRef id="44"/>
      </orthologGroup>
    </groups>
  </database>
</orthoXML>



In [208]:
gene_id_name["sp|A0A089QKZ7|Y155A_MYCTU"]



6

In [ ]:
attrib

In [123]:
for movie in root.iter('species'):
    print(movie.attrib)

In [125]:
root.iter("")

In [14]:

#import lxml.etree as etree

xml_data = "<root>...</root>"

print(canonicalize(xml_data))

NameError: name 'canonicalize' is not defined

In [ ]:

# modifying tree
for rank in root.iter('rank'):
    new_rank = int(rank.text) + 1
    rank.text = str(new_rank)
    rank.set('updated', 'yes')

tree.write('output.xml')

In [ ]:

>>> b = ET.SubElement(a, 'b')
>>> c = ET.SubElement(a, 'c')
>>> d = ET.SubElement(c, 'd')
>>> ET.dump(a)
<a><b /><c><d /></c></a>

In [32]:



gene_name = ET.Element('genes')
id_val=1
protId=1
b = ET.SubElement(gene_name, "gene id=\""+str(id_val)+"\" protId=\""+str(protId)+"\"") #<gene id="9361504" protId="KRYMA19363"/>
ET.dump(gene_name)




<genes><gene id="1" protId="1" /></genes>


In [92]:

import xml.etree.ElementTree as ET

ortho_file = ET.Element("orthoXML file1") #  xmlns=\"http://orthoXML.org/2011/\
species = ET.SubElement(ortho_file, "species")

ET.dump(ortho_file)  
print("\n \n ")

print(etree.tostring(ortho_file, xml_declaration=True, encoding='utf8', pretty_print=True).decode())

# print("write it")
# et = ET.ElementTree(ortho_file)
# et.write("output_1.xml")
# !cat "output_1.xml"



<orthoXML file1><species /></orthoXML file1>

 
 


TypeError: Type 'xml.etree.ElementTree.Element' cannot be serialized.

<orthoXML xmlns="http://orthoXML.org/2011/" />

In [65]:


#print(ET.tostring(ortho_file, xml_declaration=True, encoding='utf8').decode())


In [117]:

import xml.etree.ElementTree as ET

ortho_file = ET.Element("orthoXML") # xmlns=\"http://orthoXML.org/2011/\"
species = ET.SubElement(ortho_file, "species")
species.text= "sdsd "


#for species_name in query_species_names[:5]:
    
#     species = ET.SubElement(ortho_file,"species name=\""+species_name+"\"")

    #<species name="Kryptolebias marmoratus" NCBITaxId="37003">
ET.dump(ortho_file)  

#ET.tostring(ortho_file, xml_declaration=True, encoding='utf8').decode()

# et = ET.ElementTree(ortho_file)
# et.write("output1.xml")
# # .write('output.xml')
# !cat "output1.xml"


<orthoXML><species>sdsd </species></orthoXML>


In [93]:
from lxml import etree

In [94]:
root = etree.Element("OrthoXML")

In [ ]:
import xml.etree.ElementTree as ET



In [95]:
from xml.etree import ElementTree
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [145]:
print(prettify(et))

AttributeError: type object 'ElementTree' has no attribute 'tostring'

In [110]:
import xml.etree.ElementTree as etree

from xml.etree.ElementTree import Element
from xml.etree.ElementTree import ElementTree




In [114]:
root =Element("OrthoXML")
tree=ElementTree(root)
print(etree.tostring(root))

name=Element(‘name’)

root.append(name)





b'<OrthoXML />'
